<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Rheem_Paper_Cutting_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rheem Paper Cutting Stock

## Objective and Prerequisites

This cutting stock problem shows you how to determine the optimal rolls of paper to cut with different cut patterns to satisfy various demands for different width of papers. The objective is to cut the least rolls of paper to satisfy all the demand. The objectives of the cutting stock problem are:

* Minimize the rolls of paper that must be cut,
* Make sure the cut satisfies demand for various widths, and
* Ensure that the combination of cuts are valid, i.e., must not exceed 60 inches.


---
## Problem Description

![picture](https://drive.google.com/uc?id=1Db10dmpuWmQke8iw4PDbvxdUwVoNqfHr)

The Rheem Paper Copmpany produces rolls of paper of various types for its customers. One type is produced in standard rools that are 60 inches wide and (when unwound) 200 yards long. Customers for this type of paper order rolls that are all 200 yards long, but can have any of the widths 12, 15, 20, 24, 30, or 40 inches. In a given week, Rheem waits for all orders and then decides how to cut its 60-inch rolls to satisfy the orders. For example, if there are five orders for 15-inch widths and two orders for 40-inch widths, Rheem could satisfy the order by producing three rolls, cutting each of the first two into a 40-inch and a 15-inch cut (with 5 inches left over) and cutting the third into four 15-inch cuts (with one of these left over). Each week, Rheem must decide how to cut its rolls in the most economical way to meet its orders. Specifically, it wants to cut as few rolls as possible.

The following table lists an example orders of various paper widths. 
	
| Width|	12    | 15    | 20	  | 24   | 30  | 40	 | 
| ---     | ---   | ---   | ---   | ---  | ---  | ---  | 
|Demand   | 48    | 19	| 22| 32 | 14 | 7 | 

Rheem Paper wants to determine a most efficient cutting plan to minimize the number of rolls of paper to cut.

# Discussion

A key challenge of the above problem is that there are many different possible patterns to cut, so the options to cut in various ways are quite large. 

One way to approach the problem is to recognize that we can setup a table that lists all possible cutting patterns so that we know for each patter how many rolls of different widths can be obtained. 

From a conceptual level, this problem shares some structural similarities with the employee scheduling example. In what follows, we implement the above idea.

## Model Formulation

---

### Indices

$i \in \{1..6\}$: Index to represent different widths

$j \in \{1..N\}$: Index to represent different cut patterns (in this specific example N=26)

### Parameters

$d_{i}$: Demand for width $i$


### Calculated Parameter

$A_{ij}$: Rolls of width $i$ that can be obtained from cut pattern $j$ (We show how to compute $A_{ij}$ in the python implementation below)

### Decision Variables

$x_{j}$: Number of rolls of pattern $j$ to cut


### Objective Function

- **Rolls of Paper**. We want to minimize the total rolls of papers to cut.


\begin{equation}
\text{Min}_{x_{j}} \quad \sum_{j \in \{1..N\}} x_{j}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\sum_{j \in \{1..N\}} A_{ij}*x_{j} \geq d_{i} \quad \forall i \in \{1...6\} \quad (\text{Satisfy demand for each width})
\tag{1}
\end{equation}

\begin{equation}
x_{j} \in Integer^+ \quad \forall j \in \{1..N\} \quad (\text{integer number of rolls to cut})
\tag{2}
\end{equation}


---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 6.2 MB/s 


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the inputs

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('stock cutting')

# Inputs

width = [*range(0,6)]

width_label = ['12','15','20','24','30','40']

# demand of different width
d = [48,19,22,32,14,7]

Restricted license - for non-production use only - expires 2023-10-25


Compute the $A_{ij}$ table

In [ ]:
import math
# actual width - only used for constructing the A_ij matrix
w = [12,15,20,24,30,40]

# stock width
W = 60

# compute the maximum number of width available if the stock paper is cut only into that width
bound = [math.floor(W / x) for x in w]

#print(bound)

# loop through different combinations to obtain possible number of cuts
A = []
N = 0;
for i0 in range(bound[0]+1):
  for i1 in range(bound[1]+1):
    for i2 in range(bound[2]+1):
      for i3 in range(bound[3]+1):
        for i4 in range(bound[4]+1):
          for i5 in range(bound[5]+1):

            # check whether the combination is valid
            total_w = i0*w[0] + i1*w[1] + i2*w[2] + i3*w[3] + i4*w[4] + i5*w[5];  
            
            if total_w <= W and W - total_w < w[0]:
              A_temp = [i0, i1, i2, i3, i4, i5]
              A.append(A_temp)
              N = N +1

print(np.matrix(A))   
#print(N) 

pattern = [*range(0,N)]

[[0 0 0 0 2 0]
 [0 0 0 1 1 0]
 [0 0 1 0 0 1]
 [0 0 1 0 1 0]
 [0 0 3 0 0 0]
 [0 1 0 0 0 1]
 [0 1 1 1 0 0]
 [0 1 2 0 0 0]
 [0 2 0 0 1 0]
 [0 2 0 1 0 0]
 [0 2 1 0 0 0]
 [0 4 0 0 0 0]
 [1 0 0 0 0 1]
 [1 0 0 2 0 0]
 [1 0 1 1 0 0]
 [1 0 2 0 0 0]
 [1 1 0 0 1 0]
 [1 1 0 1 0 0]
 [1 3 0 0 0 0]
 [2 0 0 0 1 0]
 [2 1 1 0 0 0]
 [2 2 0 0 0 0]
 [3 0 0 1 0 0]
 [3 0 1 0 0 0]
 [3 1 0 0 0 0]
 [5 0 0 0 0 0]]


Setup decisions, objective, and constraints

In [ ]:
# Build decision variables: whether to assign destination i to carrier j
x = m.addVars(pattern, vtype=GRB.INTEGER, name='Assign')

In [ ]:
# Objective function: Minimize total cost
m.setObjective(gp.quicksum(x[j] for j in pattern), GRB.MINIMIZE)

In [ ]:
#Constraints

# Commitment Constraints
DemandConstrs = m.addConstrs((gp.quicksum(A[j][i]*x[j] for j in pattern) >= d[i] for i in width), 
                                      name='DemandConstrs')


Solve the model

In [ ]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 6 rows, 26 columns and 53 nonzeros
Model fingerprint: 0xbc8b13b9
Variable types: 0 continuous, 26 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 5e+01]
Presolved: 6 rows, 26 columns, 53 nonzeros

Continuing optimization...


Explored 1 nodes (6 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 47 94 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.700000000000e+01, best bound 4.700000000000e+01, gap 0.0000%


Examine outputs

In [ ]:
# print optimal cut by patterns

print("\033[1m Optimal cuts by patterns [12,15,20,24,30,40]")
print("------------------------------------------\n")

# initializing pattern label
p_label = 'A'
 
# loop through all patterns
for i in pattern:
  if x[i].x > 1e-6:
    print("\033[1m Pattern",p_label,"\033[0m ",A[i],":",x[i].x,"rolls")
    p_label = chr(ord(p_label) + 1)

print("------------------------------------------")
print("The minimum rolls of paper to cut is",m.ObjVal)

 Optimal cuts by patterns [12,15,20,24,30,40]
------------------------------------------

 Pattern A   [0, 0, 0, 0, 2, 0] : 7.0 rolls
 Pattern B   [0, 0, 1, 0, 0, 1] : 7.0 rolls
 Pattern C   [0, 0, 3, 0, 0, 0] : 5.0 rolls
 Pattern D   [0, 4, 0, 0, 0, 0] : 4.0 rolls
 Pattern E   [1, 0, 0, 2, 0, 0] : 16.0 rolls
 Pattern F   [1, 3, 0, 0, 0, 0] : 1.0 rolls
 Pattern G   [3, 0, 0, 1, 0, 0] : 1.0 rolls
 Pattern H   [5, 0, 0, 0, 0, 0] : 6.0 rolls
------------------------------------------
The minimum rolls of paper to cut is 47.0


Notice that most patterns do not generate any waste, except for pattern G which generate 3 inches of waste per cut.

In [ ]:
# print obtained rolls of paper with different widths

print("\033[1m Available rolls of different widths")
print("------------------------------------------\n")

# loop through all widths
for i in width:
  available_rolls = 0
  for j in pattern:
    available_rolls += A[j][i]*x[j].x
  print("\033[1m Width",w[i],"\033[0m (requires",d[i],"): cut",available_rolls,"rolls")

 Available rolls of different widths
------------------------------------------

 Width 12  (requires 48 ): cut 50.0 rolls
 Width 15  (requires 19 ): cut 19.0 rolls
 Width 20  (requires 22 ): cut 22.0 rolls
 Width 24  (requires 32 ): cut 33.0 rolls
 Width 30  (requires 14 ): cut 14.0 rolls
 Width 40  (requires 7 ): cut 7.0 rolls


#Conclusion

The stock cutting example shows that to meet the weekly demand of various paper widths, the minimum number of rolls to cut is 47 rolls, with a combination of eight different patterns to cut. Notice that the achieved rolls of different width exceeds the actual requirement, suggesting that a perfect match is not possible to meet, and some waste is necessarily generated. However, from a planning horizon perspective, the excess rolls of paper can be used to satisfy future demand, and therefore the waste is not a concern. The only exception is that pattern G above does generate 3 inches of waste per cut. It is clearly possible to restrict the cuts to generate no waste, that is, certain patterns cannot be cut. What is your take on this aspect?  

A key take away of the above example is that the stock cutting problem can be tackled by first generate a set of possible combinations of different options to help the decision making process. This example share similar conceptual approach as the employee scheduling problem we studied earlier, where the availability information must be created based on available information, as opposed to directly provided. 

##  References

[1] Gurobi python reference. https://www.gurobi.com/documentation/

[2] This notebook is developed by Yimin Wang. If you have any comments or suggestions, please contact yimin_wang@asu.edu.